## Instalando bibliotecas necessarias

In [ ]:
!pip install Flask
!pip install pyngrok
!pip install ultralytics
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import tempfile
import os
import time


def _censor_frame(frame, yolo_model):
    """
    Função auxiliar para censurar pessoas em um único quadro (frame).
    Recebe um quadro e o modelo YOLO já carregado.
    """
    if yolo_model is None:
        print("Modelo YOLO não está carregado. Ignorando a censura.")
        return frame

    results = yolo_model.predict(source=frame, conf=0.25, verbose=False)

    detections = results[0].boxes.data
    # detections = results[0].boxes.data.numpy()
    start_detection_time = time.time()
    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        # A classe "person" no dataset COCO é 0
        if int(cls) == 0:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            y1, y2 = max(0, y1), min(frame.shape[0], y2)
            x1, x2 = max(0, x1), min(frame.shape[1], x2)

            if y1 < y2 and x1 < x2:
                person_region = frame[y1:y2, x1:x2]

                blurred_region = cv2.GaussianBlur(person_region, (71, 71), 30)

                frame[y1:y2, x1:x2] = blurred_region
    if len(detections) > 0:
      dps = len(detections)/(time.time()-start_detection_time)
      print(f"Avg. time per detection: {dps:.2f}")
    return frame

def censor_video_from_bytes(video_bytes: bytes) -> bytes:
    print("Starting censor process...")
    try:
        # model = YOLO("yolo11s.pt").to("cpu")
        model = YOLO("yolo11s.pt").to("cuda")
    except Exception as e:
        print(f"Erro ao carregar o modelo YOLO. {e}")
        model = None

    if model is None:
        raise RuntimeError("O modelo YOLO não pôde ser carregado. A função não pode continuar.")

    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_input_file:
        temp_input_file.write(video_bytes)
        input_video_path = temp_input_file.name

    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_output_file:
        output_video_path = temp_output_file.name

    try:
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            raise IOError("Não foi possível abrir o vídeo a partir dos bytes fornecidos.")

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        start_time = time.time()
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            censored_frame = _censor_frame(frame, model)

            out.write(censored_frame)
            frame_count += 1

        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time
        print(f"FPS: {fps:.2f}")
        print(f"Frames: {frame_count:.2f}")
        print(f"Elapsed time: {elapsed_time:.2f}")

        cap.release()
        out.release()

        with open(output_video_path, 'rb') as f:
            processed_video_bytes = f.read()
        return processed_video_bytes

    finally:
        if os.path.exists(input_video_path):
            os.remove(input_video_path)
        if os.path.exists(output_video_path):
            os.remove(output_video_path)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import Response
import io
from typing import Annotated
import nest_asyncio

nest_asyncio.apply()
app = FastAPI()

@app.get("/")
async def read_root():
    """
    Root endpoint that returns a simple greeting.
    """
    return {"message": "Health check: API is running!"}

@app.get("/test")
async def read_root():
    """
    Root endpoint that returns a simple greeting.
    """
    return {"message": "Test_message: API is running!"}


@app.post("/upload_video/")
async def process_video_endpoint(video_file: UploadFile = File(...)):
    video_bytes = await video_file.read()

    # Processa o vídeo
    processed_bytes = censor_video_from_bytes(video_bytes)

    return Response(content=processed_bytes, media_type="video/mp4")


In [ ]:
from pyngrok import ngrok
import os
from google.colab import userdata
import time

NGROK_AUTH_TOKEN = "NGROK_TOKEN"
if NGROK_AUTH_TOKEN:
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
else:
  print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")

port = 8000  
public_url = ngrok.connect(port, domain="known-phoenix-specially.ngrok-free.app")

print(f"Flask app exposed at: {public_url}")
time.sleep(5)

Flask app exposed at: NgrokTunnel: "https://known-phoenix-specially.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import uvicorn
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [901]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     34.213.214.55:0 - "GET / HTTP/1.1" 200 OK
INFO:     34.213.214.55:0 - "GET / HTTP/1.1" 200 OK
INFO:     34.213.214.55:0 - "GET / HTTP/1.1" 200 OK
Starting censor process...
Avg. time per detection: 302.63
Avg. time per detection: 334.24
Avg. time per detection: 507.90
Avg. time per detection: 247.86
Avg. time per detection: 278.09
Avg. time per detection: 245.36
Avg. time per detection: 229.28
Avg. time per detection: 252.83
Avg. time per detection: 300.97
Avg. time per detection: 282.47
Avg. time per detection: 226.46
Avg. time per detection: 263.16
Avg. time per detection: 237.92
Avg. time per detection: 258.58
Avg. time per detection: 269.33
Avg. time per detection: 230.29
Avg. time per detection: 232.94
Avg. time per detection: 239.35
Avg. time per detection: 200.56
Avg. time per detection: 201.93
Avg. time per detection: 238.87
Avg. time per detection: 250.26
Avg. time per detection: 226.01
Avg. time per detection: 271.46
Avg. time per detection: 245.10
Avg. time per det